In [ ]:
#!pip install -U openai-whisper
#!pip3 install googletrans==3.1.0a0
#!pip install pytube

In [ ]:
#!pip install git+https://github.com/openai/whisper.git 

In [ ]:
#!pip install git+https://github.com/pytube/pytube

In [44]:
import torch
import os

# Initialize the device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [45]:
import whisper

In [46]:
from pytube import YouTube

def video_to_audio(video_URL, destination, final_filename):
  # Get the video
  video = YouTube(video_URL)

  # Convert video to Audio
  audio = video.streams.filter(only_audio=True).first()

  # Save to destination
  output = audio.download(output_path = destination)

  _, ext = os.path.splitext(output)
  new_file = final_filename + '.mp3'

  # Change the name of the file
  os.rename(output, new_file)

In [47]:
from datetime import timedelta
import os
import whisper
import torch

def transcribe_audio(video_name):
    model = whisper.load_model("medium") # Change this to your desired model
    print("Whisper model loaded.")
    transcribe = model.transcribe(audio=video_name)
    segments = transcribe['segments']

    for segment in segments:
        startTime = str(0)+str(timedelta(seconds=int(segment['start'])))+',000'
        endTime = str(0)+str(timedelta(seconds=int(segment['end'])))+',000'
        text = segment['text']
        segmentId = segment['id']+1
        segment = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] == ' ' else text}\n\n"

        video_name = video_name.replace(".mp3","")
        srtFilename =f"{video_name}.str"
        with open(srtFilename, 'a', encoding='utf-8') as srtFile:
            srtFile.write(segment)

    return srtFilename

In [57]:
from googletrans import Translator, constants
from pprint import pprint

def translate_google(file_name):
  # init the Google API translator
    translator = Translator()

    file = open(file_name, "r")
    lines = file.readlines()
    file.close()

    traducao = []
    translations = translator.translate(lines, dest="pt")
    for translation in translations:
        if(translation!="\n"):
            traducao.append(translation.text +"\n")
        else:
            traducao.append("\n")

    texto = ""
    for palavras in traducao:
        texto = texto + str(palavras)

    with open(f"traduzido_{file_name}", 'a', encoding='utf-8') as srtFile:
        srtFile.write(str(texto))

In [58]:
video_URL = 'https://www.youtube.com/watch?v=JT3a9HCrv8w'
destination = "."
final_filename = YouTube(video_URL).title

In [59]:
video_to_audio(video_URL, destination, final_filename)

In [60]:
str_file_name = transcribe_audio(f"{final_filename}.mp3")

Whisper model loaded.


C:\Users\PEDRO\.conda\envs\openai\lib\site-packages\whisper\transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [61]:
translate_google(str_file_name)

In [62]:
def download_Video(video_URL):
    video = YouTube(video_URL)
    itag = None
    res= 0
    
    yt = video.streams.filter(adaptive=True)
    
    for stream in yt:
        try:
            if(int(stream.resolution.replace("p",""))>res):
                res=int(stream.resolution.replace("p",""))
                itag=stream.itag
        except:
            pass

    yt.get_by_itag(itag).download()

In [63]:
download_Video(video_URL)

In [64]:
import os
import re

def rename_files(final_filename):
    path = "C:/Users/PEDRO/Desktop/C"
    for filename in os.listdir(path):
        if filename.endswith('.mp3'):
            continue
        if filename.endswith('.str'):
            continue
        else:
            if final_filename.replace("'","") in filename:
                os.rename(os.path.join(path, filename), os.path.join(path, final_filename.split('.')[0] + '.mp4'))

In [65]:
rename_files(final_filename)

In [66]:
import ffmpeg

video_stream = ffmpeg.input(f'{final_filename}.mp4')
audio_stream = ffmpeg.input(f'{final_filename}.mp3')
ffmpeg.output(audio_stream, video_stream, f'VideoWith_audio_{final_filename}.mp4').run()

(None, None)

In [67]:
video_path = f'VideoWith_audio_{final_filename}.mp4'
subtitle_path = f"{final_filename}.str"
output_path = f"LegendaOriginal_{final_filename}.mp4"

In [68]:
import subprocess

try:
    subprocess.check_output(['ffmpeg', '-i', video_path, '-i', subtitle_path , '-c', 'copy', '-c:s', 'mov_text', output_path], stderr=subprocess.STDOUT)
except subprocess.CalledProcessError as e:
    print(e.output)

In [69]:
video_path = f'VideoWith_audio_{final_filename}.mp4'
subtitle_path = f"traduzido_{final_filename}.str"
output_path = f"LegendaTraduzida_{final_filename}.mp4"

In [71]:
import subprocess

try:
    subprocess.check_output(['ffmpeg', '-i', video_path, '-i', subtitle_path , '-c', 'copy', '-c:s', 'mov_text', output_path], stderr=subprocess.STDOUT)
except subprocess.CalledProcessError as e:
    print(e.output)

In [ ]:
import pygame
import sys
import random
import string

# Inicializando o Pygame
pygame.init()

# Configurações da janela
largura_tela = 800
altura_tela = 600
tela = pygame.display.set_mode((largura_tela, altura_tela))
pygame.display.set_caption("")

# Configurações da fonte
fonte_titulo = pygame.font.SysFont("Arial", 60)
fonte_palavra = pygame.font.SysFont("Arial", 40)
fonte_letras = pygame.font.SysFont("Arial", 30)

# Cores
BRANCO = (255, 255, 255)
PRETO = (0, 0, 0)

# Lista de frases
file = open("traduzido_Life could be dream monke.str", "r",encoding='utf-8')
lines = file.readlines()
file.close()


nova_lista = list(filter(lambda x: x != '\n', lines))


frases = []
minutos = []

for i in range(1,len(nova_lista),3):
    lista = str(nova_lista[i].replace("\n", "").replace(',000',""))
    inicio, fim = lista.split(" --> ")
    inicio_formatado = f"{inicio[:2]}h:{inicio[3:5]}m:{inicio[6:]}s"
    fim_formatado = f"{fim[:2]}h:{fim[3:5]}m:{fim[6:]}s"
    teste = inicio_formatado + "-->" + fim_formatado
    minutos.append(teste)

    
for i in range(2,len(nova_lista),3):
    frases.append(nova_lista[i].replace(",", "").replace(".", "").replace("\n", ""))


minuto_atual =-1
frase_atual = -1

# Variáveis de jogo
pontuacao = 0
letras_digitadas = []
letras_frase = []
letras_adivinhadas = []

# Função para preparar a próxima frase
def proxima_frase():
    global frase_atual, letras_frase, letras_adivinhadas,minuto_atual
    frase_atual += 1
    minuto_atual+=1
    letras_frase = list(frases[frase_atual].lower())
    letras_adivinhadas = ["_" if letra != " " else " " for letra in letras_frase]
    letras_digitadas.clear()

# Função para dividir a frase em linhas
def dividir_frase_em_linhas(frase, limite_largura):
    palavras = frase.split()
    linhas = []
    linha_atual = ""
    for palavra in palavras:
        if fonte_palavra.size(linha_atual + palavra + " ")[0] <= limite_largura:
            linha_atual += palavra + " "
        else:
            linhas.append(linha_atual)
            linha_atual = palavra + " "
    linhas.append(linha_atual)
    return linhas

# Preparar a primeira frase
proxima_frase()

# Loop principal do jogo
while True:
    # Eventos do Pygame
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.unicode.isalpha() and event.unicode.lower() not in letras_digitadas:
                letra = event.unicode.lower()
                letras_digitadas.append(letra)
                if letra in letras_frase:
                    for i in range(len(letras_frase)):
                        if letras_frase[i] == letra:
                            letras_adivinhadas[i] = letra
                    pontuacao += 1
                    if "_" not in letras_adivinhadas:
                        venceu_surface = fonte_titulo.render("Próxima Frase", True, BRANCO)
                        tela.blit(venceu_surface, (largura_tela // 2 - venceu_surface.get_width() // 2, 350))
                        pygame.display.update()
                        pygame.time.wait(5000)
                        if frase_atual < len(frases) - 1:
                            proxima_frase()
                        else:
                            venceu_surface = fonte_titulo.render("Você completo o jogo. Parabés.", True, BRANCO)
                            sys.exit()

                else:
                    pontuacao -= 1
                    
    tela.fill(PRETO)

    # Quebrar a frase em linhas se necessário
    linhas_frase = []
    linha_atual = ""
    for letra in letras_adivinhadas:
        if fonte_palavra.size(linha_atual + letra)[0] > largura_tela - 100:
            linhas_frase.append(linha_atual)
            linha_atual = letra
        else:
            linha_atual += letra
    linhas_frase.append(linha_atual)
    
    
    altura_minutos = fonte_letras.get_height() + 10
    for i, linha in enumerate(dividir_frase_em_linhas(minutos[minuto_atual], largura_tela - 100)):
        linha_surface = fonte_letras.render("A frase esta nesse intervalo:"+str(linha), True, BRANCO)
        tela.blit(linha_surface, (50, 100 + i * altura_minutos))

    # Desenhar as linhas da frase
    altura_linha = fonte_palavra.get_height() + 10
    for i, linha in enumerate(linhas_frase):
        linha_surface = fonte_palavra.render(linha, True, BRANCO)
        tela.blit(linha_surface, (50, 150 + i * altura_linha))
        
    # Desenhar as letras digitadas
    letras_digitadas_surface = fonte_letras.render("Letras digitadas: " + " ".join(letras_digitadas), True, BRANCO)
    tela.blit(letras_digitadas_surface, (20, 400))

    # Desenhar as letras erradas
    letras_erradas = [letra.upper() for letra in letras_digitadas if letra not in letras_frase]
    if len(letras_erradas) > 0:
        letras_erradas_surface = fonte_letras.render("Letras erradas: " + " ".join(letras_erradas), True, BRANCO)
        tela.blit(letras_erradas_surface, (20, 450))

    # Desenhar as letras corretas
    letras_corretas = [letra.upper() for letra in letras_digitadas if letra in letras_frase]
    if len(letras_corretas) > 0:
        letras_corretas_surface = fonte_letras.render("Letras corretas: "+ " " .join(letras_corretas), True, BRANCO)
        tela.blit(letras_corretas_surface, (20, 500))

    # Desenhar a pontuação
    pontuacao_surface = fonte_letras.render("Pontuação: " + str(pontuacao), True, BRANCO)
    tela.blit(pontuacao_surface, (largura_tela - pontuacao_surface.get_width() - 20, 20))

    # Atualizar a tela
    pygame.display.update()
